In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as smf
from scipy.stats  import norm, genextreme,logistic
from scipy  import stats
import scipy.stats 

In [2]:
def var_historic(data,weight,level): 
    intialvalue=np.dot(data.iloc[0,:],weight.T) 
    returns= data.pct_change().dropna()
    portfolio=  pd.DataFrame(np.dot(returns, weight.T) ) 
    
    return (np.percentile(portfolio, level))*intialvalue*np.sqrt(63)


In [3]:
def cvar_historic(data,weight,level):
    intialvalue=np.dot(data.iloc[0,:],weight.T)
    returns= data.pct_change().dropna()
    portfolio=  pd.DataFrame(np.dot(returns, weight.T) )#.sort_values(by=0)
    var=np.percentile(portfolio, level)
    cvar=portfolio[portfolio<= var].mean()
    return (cvar)*intialvalue*np.sqrt(63)


In [4]:
def var_parametric(data, weight,level):
    intialvalue=np.dot(data.iloc[0,:],weight.T)
    returns= data.pct_change().dropna()
    portfolio=  pd.DataFrame(np.dot(returns, weight.T) )
    moy=portfolio.mean()
    cov= portfolio.std()
    var=norm.ppf(level,moy,cov)
    return var*intialvalue*np.sqrt(63)


In [5]:
def ccvar_parametric(data, weight,level):
    intialvalue=np.dot(data.iloc[0,:],weight.T)
    returns= data.pct_change().dropna()
    portfolio=  pd.DataFrame(np.dot(returns, weight.T) )
    moy=portfolio.mean()
    cov= portfolio.std()
    var=norm.ppf(level,moy,cov)
    for i in range(6):
      var= var+ norm.ppf(level+0.01*i,moy,cov)

    return (1/6)*var*intialvalue*np.sqrt(63)


In [6]:
def cvar_parametric(data, weight,level):
    intialvalue=np.dot(data.iloc[0,:],weight.T)
    returns= data.pct_change().dropna()
    portfolio=  pd.DataFrame(np.dot(returns, weight.T) )
    moy=portfolio.mean()
    cov= portfolio.std()
    var=norm.ppf(level,moy,cov)
    qmc_= stats.qmc.Sobol(d=1, scramble=True).random(10000)
    N= norm.ppf(qmc_,moy,cov)
    cvar=N[N<= var].mean()
   
    return (cvar+ var)*intialvalue*np.sqrt(63)

In [23]:
def var_montecarlo_c(data,weight,level, loi):
    intialvalue=np.dot(data.iloc[0,:],weight.T)
    returns= data.pct_change().dropna()
    mu= returns.mean().to_numpy()
    sigma=np.linalg.cholesky(returns.cov().to_numpy())
    sim=100000
    portfolio=np.zeros((returns.shape[0],sim))
    for i in range(sim):
        if(loi=="normal"):
          N= np.random.normal(size=(returns.shape))
        elif(loi=='logistic'):
          N= np.random.logistic(size=(returns.shape))
        else:
          N= genextreme.rvs(0,size=(returns.shape))

        daily_returns= (mu.T + (np.dot(sigma,N.T)).T) # mu + np.inner(sigma,N)
        portfolio[:,i]=np.cumprod((np.dot(weight,daily_returns.T )+1))*intialvalue
       # plt.plot( portfolio[:,i])
    portfolio_mean=portfolio[-1,:]- intialvalue
    var=np.percentile(portfolio_mean, 1-level) 
    cvar=pd.DataFrame(portfolio_mean)[pd.DataFrame(portfolio_mean)<= var].mean()
    return   var, cvar


In [8]:
def cvar_montecarlo_(data,weight,level, loi):
    intialvalue=np.dot(data.iloc[0,:],weight.T)
    returns= data.pct_change().dropna()
    mu= returns.mean().to_numpy()
    sigma=np.linalg.cholesky(returns.cov().to_numpy())
    sim=2**12
    portfolio=np.zeros((returns.shape[0],sim))
    for i in range(sim):
        qmc_= stats.qmc.Sobol(d=returns.shape[0], scramble=True).random(returns.shape[1])
        if(loi=="normal"):
          N= norm.ppf(qmc_)
        elif(loi=='logistic'):
          N= logistic.ppf(qmc_)
        else:
          N= genextreme.ppf(qmc_)

        daily_returns= (mu.T + (np.dot(sigma,N)).T) # mu + np.inner(sigma,N)
        portfolio[:,i]=np.cumprod((np.dot(weight,daily_returns.T )+1))*intialvalue
        #plt.plot( portfolio[:,i])
    portfolio_mean=portfolio[-1,:]-intialvalue
    var=np.percentile(portfolio_mean, 1-level) 
    cvar=pd.DataFrame(portfolio_mean)[pd.DataFrame(portfolio_mean)<= var ].mean() 
    return var,cvar 
    
   

In [9]:
    
def cvar_montecarlo(data,weight,level):
    intialvalue=np.dot(data.iloc[0,:],weight.T)
    returns= data.pct_change().dropna()
    mu= returns.mean().to_numpy()
    sigma=np.linalg.cholesky(returns.cov().to_numpy())
    sim=50000
    portfolio=np.zeros((returns.shape[0],sim))
    for i in range(sim):
        N= np.random.normal(size=(returns.shape))
        daily_returns= (mu.T + (np.dot(sigma,N.T)).T) # mu + np.inner(sigma,N)
        portfolio[:,i]=np.cumprod((np.dot(daily_returns, weight.T)+1))*intialvalue
    portfolio_mean=portfolio[-1,:]
    var=np.percentile(portfolio_mean, level)-intialvalue
    cvar=pd.DataFrame(portfolio_mean)[pd.DataFrame(portfolio_mean)<= var+intialvalue].mean() 
    return  var, cvar 


In [10]:
def data_(tickers,date1,date2):
    returns=pd.DataFrame()
    for i in range(len(tickers)):
        data=yf.download(tickers[i],date1,date2)
        returns[tickers[i]]=((data["Adj Close"]).dropna())
       
    return returns

In [11]:

start_date= '2023-01-01'
end_date="2023-04-01"
tickers=["GOOGL", "MSFT", "GC=F", "TSLA",'AAPL', "BABA","^IXIC", "^TNX"]
tada=data_(tickers,start_date,end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
weight=np.array([0.07692308, 0.07692308, 0.30769231, 0.11538462, 0.26923077,
        0.03846154, 0.03846154, 0.07692308])
print(var_historic(tada,weight,0.95))
print(cvar_historic(tada,weight,0.95))
print(var_parametric(tada,weight,0.95))
print(cvar_parametric(tada,weight,0.95))




-156.37437227574898
0   -161.405081
dtype: float64
[174.10090745]
[193.62123157]


C:\Users\hp\AppData\Local\Temp\ipykernel_10644\363720332.py:8: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  qmc_= stats.qmc.Sobol(d=1, scramble=True).random(10000)


In [24]:
#cvar_parametric(tada,weight,0.95)
var_montecarlo_c(tada,weight,0.95,"normal")

(-63.12006792090168,
 0   -82.36699
 dtype: float64)

In [14]:
cvar_montecarlo_(tada,weight,0.05,"normal")

(92.13564734276616,
 0    76.451407
 dtype: float64)

In [15]:
print(var_montecarlo_c(tada,weight,0.99,'normal'))


(17.414007949587525, 0   -12.186296
dtype: float64)


In [16]:
genextreme.rvs(0,size=(5,5))

array([[ 1.93522213,  0.24222753, -0.68322932,  1.41804114,  0.82495396],
       [-1.22320782,  1.42578028, -1.04399221,  1.23640967,  1.26801362],
       [-0.62124683, -0.50023151,  3.47945922,  1.81216825,  0.47394759],
       [-0.05391303,  2.40656508,  0.87614623, -1.31815202,  1.43611106],
       [ 1.94706203,  0.04682168, -0.13807708, -0.77354947, -0.56340347]])

In [17]:
def distribution_fit(data):
 from sklearn.metrics import r2_score
 y, x = np.histogram(data, bins=100, density=True)
 x = (x + np.roll(x, -1))[:-1] / 2.0
 list_loi = ['norm', 'beta','gamma', 'pareto', 't', 'lognorm', 'invgamma', 'invgauss',  'loggamma', 'alpha', 'chi', 'chi2','rayleigh',"logistic","genextreme","weibull_max"]
 sse=[]
 rsq=[]
 best_distribution=[]

 for  i  in  range(len(list_loi)) :
    #modélisation
    loi=getattr(scipy.stats, list_loi[i])
    loi_fit=loi.fit(data)

    #parametres
    loc= loi_fit[-2]
    scale=loi_fit[-1]
    arg=loi_fit[:-2]

    #pdf
    pdf=loi.pdf(x, loc=loc,scale=scale, *arg)

    #Rmse
    sse=np.sum((y-pdf)**2)
    #R_squared
    rsq=round(r2_score(y,pdf),2)
    best_distribution.append([loi,loi_fit,sse,rsq,list_loi[i]])

 return sorted(best_distribution, key=lambda x:x[2])[0]

In [25]:
def generate_distribution(dist, params, size=10):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = round(params[-2],2)
    scale = round(params[-1], 2)

    return dist.rvs(loc=loc, *arg,scale=scale, size=size ) if arg else dist.rvs(loc=loc, scale=scale, size=size)

In [26]:
def var_montecarlo(data,weight,level):
    intialvalue=np.dot(data.iloc[0,:],weight.T)
    returns= data.pct_change().dropna()
    distribution=[]
    for i in range(returns.shape[1]):
      distribution.append(distribution_fit(returns.iloc[i]))
    mu= returns.mean().to_numpy()
    sigma=np.linalg.cholesky(returns.cov().to_numpy())
    sim=1000
    portfolio=np.zeros((returns.shape[0],sim))
    N=np.ones((returns.shape[1],returns.shape[0]))
    for i in range(sim):
      for j in range(returns.shape[1]):
        N[j]=generate_distribution(distribution[j][0], distribution[j][1], returns.shape[0])
      daily_returns= (mu.T + (np.dot(sigma,N)).T) # mu + np.inner(sigma,N)
      portfolio[:,i]= np.cumprod((np.dot(weight, daily_returns.T )+1))*intialvalue #daily_returns.T
    portfolio_mean=portfolio[-1,:]- intialvalue
    var=np.percentile(portfolio_mean, 1-level) 
    cvar=portfolio_mean[portfolio_mean<= np.percentile(portfolio_mean, level)  ].mean()
    return  var, cvar
print(var_montecarlo(tada,weight,0.95))

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_continuous_distns.py:707: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_distn_infrastructure.py:2781: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_distn_infrastructure.py:2781: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_distn_infrastructure.py:2781: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_distn_infrastructure.py:2781: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
c:\Users\h

(302.1865285537912, 321.7126073403801)


In [27]:
returns= tada.pct_change().dropna()

distribution=[]
for i in range(returns.shape[1]):
      distribution.append(distribution_fit(returns.iloc[i]))

N=np.ones((returns.shape[1],returns.shape[0]))
for i in range(returns.shape[1]):
 N[i]=generate_distribution(distribution[i][0], distribution[i][1], returns.shape[0])
N

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_continuous_distns.py:707: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_distn_infrastructure.py:2781: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_distn_infrastructure.py:2781: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_distn_infrastructure.py:2781: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\stats\_distn_infrastructure.py:2781: RuntimeWarning: invalid value encountered in multiply
  Lhat = muhat - Shat*mu
c:\Users\h

array([[-1.88383413e-02, -1.89276846e-02, -3.99612514e-02,
         1.48279579e+00,  7.60957549e-02, -3.99475543e-02,
        -3.99998983e-02, -3.68067490e-02, -1.68041118e-02,
         5.89153867e-02,  3.51169155e-01,  4.15610991e-02,
         2.78999704e+00,  1.72840835e+00, -2.05697339e-02,
         8.16924440e-02, -1.82661827e-02,  5.23314384e-01,
        -3.66676123e-02, -3.99996061e-02,  3.18418858e-03,
        -2.60620180e-02,  4.89782893e-01,  6.78292760e-02,
         1.42322181e-01,  7.86667148e-01, -3.99949484e-02,
         6.20929120e-02, -3.40388036e-02, -3.63035085e-02,
        -3.97899169e-02, -3.99999578e-02, -3.99998915e-02,
        -3.28971896e-02, -3.99981601e-02,  1.14075487e+00,
        -3.99928168e-02,  1.08961963e+00, -2.57467037e-02,
        -3.79385659e-02,  4.04059901e-01,  3.21186637e+00,
         1.31356340e-01,  8.95533413e-01, -3.86265777e-02,
         1.38806871e-01, -3.36315849e-02, -3.99572751e-02,
        -3.11545833e-02, -3.99709057e-02, -3.99999998e-0